# USPTO Analysis
_______________

### Patents are dense
- Patents are inherently dense. Typically, the information may be understood with enough time, or preexisting domain knowledge. 

### Patent breakdown
- With each patent comes a set of pages; title page, description pages, graph pages, etc. Not all, but a lot of patents, come with a patent drawing/figure on the front page. Within that image, there are numbers to label each part of the figure; the aforementioned description pages explain what each image/figure is and what each given label on the image represents. 

### Project purpose
- The purpose of this project is to use machine learning and other data science methods to identify each image label, match that label with its' corresponding description, and convert the description to laymens terms as well; as well as condense that information. 

### Extra steps
- After this is done an additional step may be taken to take the condensed descriptions and place them next to each figure label. After the descriptions are placed, a search engine/recommendation engine may be created to allow for quicker access. 

### Benefit
- An individual can save time by searching and finding relavant patents based on either laymens terms or domain jargon. They can then gain a quicker oversite by receiving only the images with their descriptions; with the option of looking at the full paper. 

### Where did this idea come from?
- This project has grown out of a personal desire to understand patents, and feeling overwhelmed by the amount of data that there is. The hope is to make patents less of a black blox, and more of an opportunity for myself and for others. 

### Limitations
- Due to limited time, this presentation/final product for this bootcamp will only be a part of the overall project. 

## Prepare and organize your workspace
_______________
#### Below is a basic breakdown of what an image recognition project using tensorflow may look like. It's important to stay organized and keep things simple. 

```bash
Tensorflow/scripts/
└── preprocessing
    ├── generate_tfrecord.py
    └── xml_to_csv.py
```
```bash
Tensorflow/workspace/
└── patent_project
    ├── annotations
    │   ├── function_test.txt
    │   ├── function_train.txt
    │   ├── get_labels.ipynb
    │   ├── labelmap.pbtxt
    │   ├── test_labels.csv
    │   ├── testing.txt
    │   ├── train.record
    │   ├── train_labels.csv
    │   └── training.txt
    ├── images
    │   ├── tiff_to_jpg.ipynb
    │   ├── test 
    │   │   ├── tif files
    │   │   ├── xml files
    │   └── train
    │       ├── tif files
    │       └── xml files
    ├── pre-trained-model
    │   └── ssd_inception_v2_coco
    │       ├── checkpoint
    │       ├── frozen_inference_graph.pb
    │       ├── model.ckpt.data-00000-of-00001
    │       ├── model.ckpt.index
    │       ├── model.ckpt.meta
    │       ├── pipeline.config
    │       └── saved_model
    │           ├── saved_model.pb
    │           └── variables
    ├── train.py
    └── training
        ├── checkpoint
        ├── events.out.tfevents
        ├── model.ckpt-0.data-00000-of-00001
        ├── model.ckpt-0.index
        ├── model.ckpt-0.meta
        └── ssd_inception_v2_coco.config
```
- Set path when new terminal is opened. 
- !export PYTHONPATH=$PYTHONPATH:/Users/home/Documents/TensorFlow/models/research:/Users/home/Documents/TensorFlow/models/research/slim

## Convert images to correct format
_______________
#### The images that I downloaded in bulk from the uspto website came in a tiff format. Tensorflow doesn't support this so we need to convert it to a format such as jpg.

```
import os
from PIL import Image

path = '/Users/home/Documents/Tensorflow/workspace/patent_project/images/'
for root, dirs, files in os.walk(path, topdown=False):
    for name in files:
        print(os.path.join(root, name))
        if os.path.splitext(os.path.join(root, name))[1].lower() == ".tif":
            if os.path.isfile(os.path.splitext(os.path.join(root, name))[0] + ".jpg"):
                print("A jpg file already exists for %s" % name)
            # If a jpeg is *NOT* present, create one from the tiff.
            else:
                outfile = os.path.splitext(os.path.join(root, name))[0] + ".jpg"
                try:
                    im = Image.open(os.path.join(root, name))
                    print("Generating jpeg for %s" % name)
                    im.thumbnail(im.size)
                    im.save(outfile, "JPEG", quality=100)
                except Exception as e:
                    print(e)
```

## Annotate images
_______________
#### Using labelImg you can annotate your images by placing bounding boxes around the items that you want to identify, this then creates an xml file for each image that has the bounding box boundries for each image and its' item. 

#### Once we have our xml files we can convert them into a single csv file:

```
import os
import glob
import pandas as pd
import argparse
import xml.etree.ElementTree as ET

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (
                    root.find('filename').text, int(root.find('size')[0].text),
                    int(root.find('size')[1].text), member[0].text,
                    int(member[4][0].text), int(member[4][1].text),
                    int(member[4][2].text), int(member[4][3].text)
                    )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

def main():
    parser = argparse.ArgumentParser(description="XML-to-CSV")
    parser.add_argument("-i", "--inputDir", help="Path to xml folder", type=str)
    parser.add_argument("-o", "--outputFile", help="Output path to csv folder", type=str)
    args = parser.parse_args()
    if(args.inputDir is None):
        args.inputDir = os.getcwd()
    if(args.outputFile is None):
        args.outputFile = args.inputDir + "/labels.csv"
    assert(os.path.isdir(args.inputDir))
    xml_df = xml_to_csv(args.inputDir)
    xml_df.to_csv(args.outputFile, index=None)
    print('Converted xml to csv.')

if __name__ == '__main__':
    main()
```    

## Get labels
_______________
#### Due to the number of total possible labels that we could have for our patent images it's best to create a script to turn our csv labels into our .pbtxt items.

#### - First we create a txt file with all of the image labels: -

```
import pandas as pd

# Grab Training labels
filename = 'train_labels.csv'
file = pd.read_csv(filename,header=None)
file = file[3]
end = '\n'
s = ' '
ID = 1
name = 'training.txt'
for x in file[1:]:
    out = ''
    out += 'item' + s + '{' + end
    out += s*2 + 'id:' + ' ' + (str(ID)) + end
    out += s*2 + 'name:' + ' ' + '\'' + str(x) + '\'' + end
    out += '}' + end    
    ID += 1
    with open(name, 'a') as f:
        f.write(out)
        
# Grab Testing labels
filename = 'test_labels.csv'
file = pd.read_csv(filename,header=None)
file = file[3]
end = '\n'
s = ' '
ID = ID
name = 'testing.txt'
for x in file[1:]:
    out = ''
    out += 'item' + s + '{' + end
    out += s*2 + 'id:' + ' ' + (str(ID)) + end
    out += s*2 + 'name:' + ' ' + '\'' + str(x) + '\'' + end
    out += '}' + end    
    ID += 1
    with open(name, 'a') as f:
        f.write(out)     
```

#### - Then we our if else statement using the .txt to create a our tfrecords file: -

```
# generate_tfrecord if else statement
# Testing
filename = 'test_labels.csv'
file = pd.read_csv(filename,header=None)
file = file[3]
end = '\n'
s = ' '
ID = 0
name = 'function_test.txt'
for x in file[2:]:
    out = ''
    out += 'elif row_label == \'' + str(x) + '\':' + end
    out += s*4 + 'return ' + str(ID) + end
    ID += 1
    with open(name, 'a') as f:
        f.write(out)
        
# Training
filename = 'train_labels.csv'
file = pd.read_csv(filename,header=None)
file = file[3]
end = '\n'
s = ' '
ID = ID
name = 'function_train.txt'
for x in file[2:]:
    out = ''
    out += 'elif row_label == \'' + str(x) + '\':' + end
    out += s*4 + 'return ' + str(ID) + end
    ID += 1
    with open(name, 'a') as f:
        f.write(out)
```

#### - From the above code we can copy and paste the results into the 'class_text_to_int()' function below -

```
from __future__ import print_function
from __future__ import absolute_import
import os
import io
import pandas as pd
import tensorflow as tf
import sys
sys.path.append("../../models/research")

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

flags = tf.app.flags
flags.DEFINE_string('csv_input', '', 'Path to the CSV input')
flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
flags.DEFINE_string('label', '', 'Name of class label')
flags.DEFINE_string('img_path', '', 'Path to images')
FLAGS = flags.FLAGS

def class_text_to_int(row_label):
    if row_label == 'patent label 1':
        return 1
    elif row_label == 'patent label 2':
        return 2
    elif row_label == 'patent label 120':
        return 120
    elif row_label == 'patent label etc':
        return etc
    return None

def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    # check if the image format is matching with your images.
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example
def main(_):
    writer = tf.python_io.TFRecordWriter(FLAGS.output_path)
    path = os.path.join(os.getcwd(), FLAGS.img_path)
    examples = pd.read_csv(FLAGS.csv_input)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(), FLAGS.output_path)
    print('Created TFRecords: {}'.format(output_path))

if __name__ == '__main__':
    tf.app.run()
```

## Transfer Learning
_______________
#### Tensorflow comes with a handful of models that are pretrained. We're actually able to take any of the pretrained models, take the calculated weights for any of the given models, and use those weights to train the set of annotated images that we created. 

#### I decided to go with the 'ssd_inception_v2_coco' model simply because it seemed like a good middle ground for speed and accuracy.

#### Once we have downloaded our pretrained model we can customize the config file to the number of files to files that we have creating up to this point:

```
model {
  ssd {
    num_classes: 250 # this is the number of different labels that we came across when annotating our patent images.
    image_resizer {
      fixed_shape_resizer {
        height: 300
        width: 300
      }
    }
    feature_extractor {
      type: "ssd_inception_v2"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.99999989895e-05
          }
        }
        initializer {
          truncated_normal_initializer {
            mean: 0.0
            stddev: 0.0299999993294
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.999700009823
          center: true
          scale: true
          epsilon: 0.0010000000475
          train: true
        }
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    box_predictor {
      convolutional_box_predictor {
        conv_hyperparams {
          regularizer {
            l2_regularizer {
              weight: 3.99999989895e-05
            }
          }
          initializer {
            truncated_normal_initializer {
              mean: 0.0
              stddev: 0.0299999993294
            }
          }
          activation: RELU_6
        }
        min_depth: 0
        max_depth: 0
        num_layers_before_predictor: 0
        use_dropout: false
        dropout_keep_probability: 0.800000011921
        kernel_size: 3
        box_code_size: 4
        apply_sigmoid_to_scores: false
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.20000000298
        max_scale: 0.949999988079
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
        aspect_ratios: 3.0
        aspect_ratios: 0.333299994469
        reduce_boxes_in_lowest_layer: true
      }
    }
    post_processing {
      batch_non_max_suppression {
        score_threshold: 0.300000011921
        iou_threshold: 0.600000023842
        max_detections_per_class: 100
        max_total_detections: 100
      }
      score_converter: SIGMOID
    }
    normalize_loss_by_num_matches: true
    loss {
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      classification_loss {
        weighted_sigmoid {
        }
      }
      hard_example_miner {
        num_hard_examples: 3000
        iou_threshold: 0.990000009537
        loss_type: CLASSIFICATION
        max_negatives_per_positive: 3
        min_negatives_per_image: 0
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
  }
}
train_config {
  batch_size: 24 # Good friends don't let you do batch sizes over 32. 
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    ssd_random_crop {
    }
  }
  optimizer {
    rms_prop_optimizer {
      learning_rate {
        exponential_decay_learning_rate {
          initial_learning_rate: 0.00400000018999
          decay_steps: 800720
          decay_factor: 0.949999988079
        }
      }
      momentum_optimizer_value: 0.899999976158
      decay: 0.899999976158
      epsilon: 1.0
    }
  }
  fine_tune_checkpoint: "pre-trained-model/ssd_inception_v2_coco_2018_01_28/model.ckpt"
  from_detection_checkpoint: true
  num_steps: 200000
}
train_input_reader {
  label_map_path: 'annotations/labelmap.pbtxt'
  tf_record_input_reader {
    input_path: 'annotations/train.record'
  }
}
eval_config {
  num_examples: 8000
  max_evals: 10
  use_moving_averages: false
}
eval_input_reader {
  label_map_path: 'annotations/labelmap.pbtxt'
  shuffle: false
  num_readers: 1
  tf_record_input_reader {
    input_path: 'annotations/test.record'
  }
}
```

## Training our model
_______
#### once we have all of the above code we can run a few commands to run all of the code and train our model: 

In [28]:
# - Create xml to csv train data:
!python /Users/home/Documents/Tensorflow/scripts/preprocessing/xml_to_csv.py \
-i /Users/home/Documents/Tensorflow/workspace/training_demo/images/train \
-o /Users/home/Documents/Tensorflow/workspace/training_demo/annotations/train_labels.csv

# - Create xml to csv test data:
!python /Users/home/Documents/Tensorflow/scripts/preprocessing/xml_to_csv.py \
-i /Users/home/Documents/Tensorflow/workspace/training_demo/images/test \
-o /Users/home/Documents/Tensorflow/workspace/training_demo/annotations/test_labels.csv

# - Create record train data:
!python \
/Users/home/Documents/Tensorflow/scripts/preprocessing/generate_tfrecord.py \
--label=patent \
--csv_input=/Users/home/Documents/Tensorflow/workspace/training_demo/annotations/train_labels.csv\
--img_path=/Users/home/Documents/Tensorflow/workspace/training_demo/images/train \
--output_path=/Users/home/Documents/Tensorflow/workspace/training_demo/annotations/train.record

# - Create record test data:
!python /Users/home/Documents/Tensorflow/scripts/preprocessing/generate_tfrecord.py \
--label=patent \
--csv_input=/Users/home/Documents/Tensorflow/workspace/training_demo/annotations/test_labels.csv \
--img_path=/Users/home/Documents/Tensorflow/workspace/training_demo/images/test \
--output_path=/Users/home/Documents/Tensorflow/workspace/training_demo/annotations/test.record

# - Train model using model_main.py
!python /Users/home/Documents/Tensorflow/workspace/training_demo/model_main.py \
--logtostderr \
--train_dir=/Users/home/Documents/Tensorflow/workspace/training_demo/training/ \
--pipeline_config_path=/Users/home/Documents/Tensorflow/workspace/training_demo/training/ssd_inception_v2_coco.config


Successfully converted xml to csv.
Successfully converted xml to csv.
Successfully created the TFRecords: /Users/home/Documents/Tensorflow/workspace/training_demo/annotations/train.record
Successfully created the TFRecords: /Users/home/Documents/Tensorflow/workspace/training_demo/annotations/test.record


#### Below is an example output for our model. This shows varying sizes of our model, and what the Ave Precision & Ave Recall are. These two metrics look at the data that is relavant(our positives).

- Ave Precision: True Positive / Actual Results <br />
- Ave Recall: True Positive / Predicted Results

```
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.590
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.861
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.694
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.615
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.614
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.614
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.614
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.638
```

#### Once we run our model an event file will be created. This is where we can evaluate our model. 

In [ ]:
# - Tensorboard Allows us to see different aspects of our data.
!tensorboard --logdir=/Users/home/Documents/Tensorflow/workspace/training_demo/training/

TensorBoard 1.14.0 at http://blazes-mbp.ad.uvu.edu:6006/ (Press CTRL+C to quit)
